In [1]:
from google.colab import drive
drive.mount("/content/gdrive", force_remount = True)

Mounted at /content/gdrive


In [2]:
!pip install flask-ngrok

In [3]:
!mkdir -p /drive/ngrok-ssh
%cd /drive/ngrok-ssh
!wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip -O ngrok-stable-linux-amd64.zip
!unzip -u ngrok-stable-linux-amd64.zip
!cp /drive/ngrok-ssh/ngrok /ngrok
!chmod +x /ngrok

/drive/ngrok-ssh
--2022-04-19 09:37:36--  https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
Resolving bin.equinox.io (bin.equinox.io)... 52.202.168.65, 54.237.133.81, 18.205.222.128, ...
Connecting to bin.equinox.io (bin.equinox.io)|52.202.168.65|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 13832437 (13M) [application/octet-stream]
Saving to: ‘ngrok-stable-linux-amd64.zip’

ngrok-stable-linux- 100%[===================>]  13.19M  3.51MB/s    in 4.9s    

2022-04-19 09:37:41 (2.69 MB/s) - ‘ngrok-stable-linux-amd64.zip’ saved [13832437/13832437]

Archive:  ngrok-stable-linux-amd64.zip
  inflating: ngrok                   


In [4]:
!/ngrok authtoken 26EUUR0GrMZSuscYNumIYtVGRr2_5KKCdnmdfXx22QibUYSKn

Authtoken saved to configuration file: /root/.ngrok2/ngrok.yml


In [5]:
pip install wfdb

     |████████████████████████████████| 137 kB 7.4 MB/s 
     |████████████████████████████████| 11.2 MB 48.4 MB/s 
     |████████████████████████████████| 900 kB 50.9 MB/s 
  Attempting uninstall: matplotlib
    Found existing installation: matplotlib 3.2.2
    Uninstalling matplotlib-3.2.2:
      Successfully uninstalled matplotlib-3.2.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
albumentations 0.1.12 requires imgaug<0.2.7,>=0.2.5, but you have imgaug 0.2.9 which is incompatible.


In [6]:
pip install neurokit2

     |████████████████████████████████| 1.1 MB 6.5 MB/s 


In [7]:
from flask_ngrok import run_with_ngrok
from flask import Flask, request, render_template
import pandas as pd
import tensorflow as tf
import keras
from keras.models import load_model
import wfdb
import numpy as np
from scipy.signal import butter, filtfilt

In [8]:
app = Flask(__name__)
run_with_ngrok(app)

In [9]:
# app = Flask(__name__)
path = r"/content/gdrive/My Drive/arrhythmia/"
import sys

@app.route('/', methods=['GET', 'POST'])
def main():
    arrhythmic = 0
    condition = ""
    ratio = 0
    output = {}
    
    # If a form is submitted
    if request.method == "POST":

        myfiledat = request.files["myfiledat"]
        myfilehea = request.files["myfilehea"]


        myfiledat.save(myfiledat.filename)
        myfilehea.save(myfilehea.filename)



        file_name_array = myfiledat.filename.split('.')
        record = wfdb.rdrecord(file_name_array[0])

        model = load_model("splitonpatients.h5")
        # record = wfdb.rdrecord("100")

        def resamplerecord(signal, frequency, targetfrequency):
            from wfdb import processing
            ratio = targetfrequency / frequency
            if ratio == 1.0:
              return signal

            newsignal = []
            channels = np.shape(signal)[1]
            for channel in range(channels):
              ns, _ = wfdb.processing.resample_sig(signal[:,channel], frequency, targetfrequency)
              newsignal.append(ns)
            

            return np.column_stack(newsignal)
          

        def filtersignal(signal, order, cutoff, sample_rate):
            nyquist_rate = 0.5 * sample_rate
            lowcut, highcut = cutoff[0] / nyquist_rate, cutoff[1] / nyquist_rate
            b, a = butter(order, [lowcut,highcut], btype = 'band')

            newsignal = []
            channels = np.shape(signal)[1]
            for channel in range(channels):
              ns = filtfilt(b, a, signal[:,channel])
              newsignal.append(ns)

            return np.column_stack(newsignal)

        
        from sklearn.preprocessing import StandardScaler
        import neurokit2 as nk


        frequency = record.fs
        sequences = []

        record.p_signal = resamplerecord(record.p_signal, frequency, 360)
        record.p_signal = filtersignal(record.p_signal, order = 5, cutoff = [0.5,15], sample_rate = 360)

        scaler = StandardScaler()
        record.p_signal = scaler.fit_transform(record.p_signal)

        _, rpeaks = nk.ecg_peaks(record.p_signal[:,0], sampling_rate = 360)
        rpeaks = rpeaks['ECG_R_Peaks'].tolist()

        window = 256
        for i, sample in enumerate(rpeaks):
            sequence = np.array([])
            start, end = sample - window // 2, sample + window // 2
            
            if 0 < start < end < record.p_signal.shape[0]:
            
              sequence = record.p_signal[start:end, 0]
              sequence.reshape(1, -1, 1)

              if (sequence.size > 0) :
                sequences.append(sequence)

        x = np.vstack(sequences)
        x = np.reshape(x, (x.shape[0], x.shape[1], 1))
        predictions = model.predict(x)

        
        normal = 0
        
        for (x,y), value in np.ndenumerate(predictions):
          # print(value)
          if(value >= 0.5):
            arrhythmic += 1
          else:
            normal += 1

        print(f"Hello world! {arrhythmic}", file=sys.stderr)
        print("Arrhythmic", arrhythmic)

        print("Normal", normal)

        ratio = arrhythmic/(arrhythmic+normal)
        print("Ratio", ratio)


        if(ratio < 0.001):
          condition = "No arrhythmia detected"
        elif (ratio < 0.002):
          condition = "Mild"
        elif (ratio < 0.005):
          condition = "Moderate"
        elif (ratio < 0.007):
          condition = "Severe"
        else :
          condition = "Critical"

        print("Condition", condition)

        # Get values through input bars
        
        
        # weight = request.form.get("weight")
        
        # Put inputs to dataframe
        # X = pd.DataFrame([[height, weight]], columns = ["Height", "Weight"])
        
        # Get prediction
        output = {"arrhythmic": arrhythmic, "condition": condition}
        print(output)
        
    else:
        output = {"arrhythmic": arrhythmic, "arrythmic": condition}
        # arrhythmic = 0
        
    return render_template("website.html", output = output)

# Running the app


In [10]:
app.run()


 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


 * Running on http://3571-34-125-20-12.ngrok.io
 * Traffic stats available on http://127.0.0.1:4040


127.0.0.1 - - [19/Apr/2022 09:40:14] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [19/Apr/2022 09:40:14] "GET /static/css/style.css HTTP/1.1" 200 -
127.0.0.1 - - [19/Apr/2022 09:40:15] "GET /favicon.ico HTTP/1.1" 404 -
Hello world! 8
127.0.0.1 - - [19/Apr/2022 09:41:00] "POST / HTTP/1.1" 200 -


Arrhythmic 8
Normal 2263
Ratio 0.0035226772346983706
Condition Moderate
{'arrhythmic': 8, 'condition': 'Moderate'}
